# FRAGMENT MOLECULAR ORBITAL APPROACH
# Cutoffs for Hartree-Fock and correlation energy

In [ ]:
import pandas as pd

In [ ]:
# Function to display datas side by side
from IPython.display import display_html
def display_side_by_side(*args):
    html_str=''
    for df in args:
        html_str+=df.to_html()
    display_html(html_str.replace('table','table style="display:inline"'),raw=True)

## Register the input datas

* _Read the FMO2 datas into a dataframe_<br/>
  _Rename some columns for working with datas in a more easier way_

In [ ]:
fmo2_file = "bz-p1-uc1-fmo2-vtz.csv"
fmo2_data = pd.read_csv(fmo2_file)
fmo2_data = pd.DataFrame(fmo2_data)
fmo2_data = fmo2_data.rename(columns = {"E\"IJ-E\"I-E\"J": "mp2", "corr/uncorr": "hf", "dDIJ*VIJ,unc": "coul_bath"})

* _Read the FMO3 datas into a dataframe_<br/>
  _Rename some columns for working with datas in a more easier way_

In [ ]:
fmo3_file = "bz-p1-uc1-fmo3-vtz.csv"
fmo3_data = pd.read_csv(fmo3_file)
fmo3_data = pd.DataFrame(fmo3_data)
fmo3_data = fmo3_data.rename(columns = {"deltaE\"IJK": "mp2", "corr/uncorr": "hf", "dDIJK*VIJK": "coul_bath"})

## Register the output datas

In [ ]:
# Name the files for saving large fmo2 and fmo3 data tables
fmo2_outfile = "new-" + fmo2_file
fmo3_outfile = "new-" + fmo3_file

# TWO-BODY Effect

* _Calculate the two-body energies of each dimer_<br/>
  _Then save them to a new column E-xxx-delFMO2 in the data frame_

* _Determine the benchmark value (Ebm) for the HF, MP2 and dispersion energies of the two-body interaction ($\Delta$FMO2)_

In [ ]:
E_HF_delFMO2 = [ row.hf + row.coul_bath for index, row in fmo2_data.iterrows() ]
E_MP2_delFMO2 = [ row.mp2 + row.coul_bath for index, row in fmo2_data.iterrows() ]
E_disp_delFMO2 = [ row.mp2 - row.hf for index, row in fmo2_data.iterrows() ]

fmo2_data["E_HF_delFMO2"] = E_HF_delFMO2
fmo2_data["E_MP2_delFMO2"] = E_MP2_delFMO2
fmo2_data["E_disp_delFMO2"] = E_disp_delFMO2

Ebm_HF_delFMO2 = fmo2_data["E_HF_delFMO2"].sum()
Ebm_MP2_delFMO2 = fmo2_data["E_MP2_delFMO2"].sum()
Ebm_disp_delFMO2 = fmo2_data["E_disp_delFMO2"].sum()

In [ ]:
# Save the fmo2_data
fmo2_data.to_csv(fmo2_outfile)

* _Find the R threshold values out of the R column in the data frame_
* _For each R value in the threshold list, sum the two-body energies if this energy is in the range of the R value_ <br/>
  _Then, find the difference of this energy to the benchmark value (in kJ/mol)_

In [ ]:
R_thres = fmo2_data["R"].drop_duplicates().sort_values()

E_HF_delFMO2_in_R = []
E_MP2_delFMO2_in_R = []
E_disp_delFMO2_in_R = []

for i in R_thres:
    
    E_HF_in_R = 0.0
    E_MP2_in_R = 0.0
    E_disp_in_R = 0.0
    
    for index, row in fmo2_data.iterrows():
        if row.R <= i:
            E_HF_in_R += row.E_HF_delFMO2
            E_MP2_in_R += row.E_MP2_delFMO2
            E_disp_in_R += row.E_disp_delFMO2
    
    E_HF_delFMO2_in_R.append(E_HF_in_R)
    E_MP2_delFMO2_in_R.append(E_MP2_in_R)
    E_disp_delFMO2_in_R.append(E_disp_in_R)

diff_EHF = []
diff_EMP2 = []
diff_Edisp = []

for j in E_HF_delFMO2_in_R:
    diff_HF = (j - Ebm_HF_delFMO2)*2625.499638
    diff_HF = '{:.3f}'.format(diff_HF)
    diff_EHF.append(diff_HF)

for k in E_MP2_delFMO2_in_R:
    diff_MP2 = (k - Ebm_MP2_delFMO2)*2625.499638
    diff_MP2 = '{:.3f}'.format(diff_MP2)
    diff_EMP2.append(diff_MP2)
    
for l in E_disp_delFMO2_in_R: 
    diff_disp = (l - Ebm_disp_delFMO2)*2625.499638
    diff_disp = '{:.3f}'.format(diff_disp)
    diff_Edisp.append(diff_disp)
    
fmo2_cutoff_data = pd.DataFrame({"R": R_thres, "diff_EHF": diff_EHF, "diff_EMP2": diff_EMP2, "diff_EDisp": diff_Edisp})

# Sort cutoff data table by R from large to small values
fmo2_cutoff_data = fmo2_cutoff_data.sort_values(by=["R"], ascending=False)

##### Define a function to find the R cutoff value according to the energy criterium

In [ ]:
########################################################
# ener_type = "hf", "mp2", "disp"
def find_R_NuDimer_cutoff(full_data, cutoff_data, energy_type, crit_value):
    dict_R_NuDimer = {}
    
    if energy_type == "hf":
        R = []
        for index, row in cutoff_data.iterrows():
            if abs(float(row.diff_EHF)) > crit_value: 
                break 
            R.append(row.R)

        R_cutoff = R[-1]
        
        i = 0
        for index, row in full_data.iterrows():
            if row.R <= R_cutoff:
                i += 1        
        
    if energy_type == "mp2":
        R = []
        for index, row in cutoff_data.iterrows():
            if abs(float(row.diff_EMP2)) > crit_value: 
                break 
            R.append(row.R)

        R_cutoff = R[-1]
    
        i = 0
        for index, row in full_data.iterrows():
            if row.R <= R_cutoff:
                i += 1        
    
    if energy_type == "disp":
        R = []
        for index, row in cutoff_data.iterrows():
            if abs(float(row.diff_EDisp)) > crit_value: 
                break 
            R.append(row.R)

        R_cutoff = R[-1]
        
        i = 0
        for index, row in full_data.iterrows():
            if row.R <= R_cutoff:
                i += 1 
    
    dict_R_NuDimer["Title"] = "FMO2-" + energy_type + "-" + str(crit_value)
    dict_R_NuDimer["Rcutoff"] = R_cutoff
    dict_R_NuDimer["NuDimer"] = i
    
    return dict_R_NuDimer
########################################################
hf = "hf"
mp2 = "mp2"
disp = "disp"

#### Display Cutoff Values and the Number of Dimers within these Cutoffs

In [ ]:
hf_1_0 = find_R_NuDimer_cutoff(fmo2_data, fmo2_cutoff_data, hf, 1.0)
hf_0_5 = find_R_NuDimer_cutoff(fmo2_data, fmo2_cutoff_data, hf, 0.5)
hf_0_1 = find_R_NuDimer_cutoff(fmo2_data, fmo2_cutoff_data, hf, 0.1)
mp2_1_0 = find_R_NuDimer_cutoff(fmo2_data, fmo2_cutoff_data, mp2, 1.0)
mp2_0_5 = find_R_NuDimer_cutoff(fmo2_data, fmo2_cutoff_data, mp2, 0.5)
mp2_0_1 = find_R_NuDimer_cutoff(fmo2_data, fmo2_cutoff_data, mp2, 0.1)
disp_1_0 = find_R_NuDimer_cutoff(fmo2_data, fmo2_cutoff_data, disp, 1.0)
disp_0_5 = find_R_NuDimer_cutoff(fmo2_data, fmo2_cutoff_data, disp, 0.5)
disp_0_1 = find_R_NuDimer_cutoff(fmo2_data, fmo2_cutoff_data, disp, 0.1)


In [46]:
fmo2_cutoff_table = []
fmo2_cutoff_table.append(find_R_NuDimer_cutoff(fmo2_data, fmo2_cutoff_data, hf, 1.0))
fmo2_cutoff_table.append(find_R_NuDimer_cutoff(fmo2_data, fmo2_cutoff_data, hf, 0.5))
fmo2_cutoff_table.append(find_R_NuDimer_cutoff(fmo2_data, fmo2_cutoff_data, hf, 0.1))
fmo2_cutoff_table.append(find_R_NuDimer_cutoff(fmo2_data, fmo2_cutoff_data, mp2, 1.0))
fmo2_cutoff_table.append(find_R_NuDimer_cutoff(fmo2_data, fmo2_cutoff_data, mp2, 0.5))
fmo2_cutoff_table.append(find_R_NuDimer_cutoff(fmo2_data, fmo2_cutoff_data, mp2, 0.1))
fmo2_cutoff_table.append(find_R_NuDimer_cutoff(fmo2_data, fmo2_cutoff_data, disp, 1.0))
fmo2_cutoff_table.append(find_R_NuDimer_cutoff(fmo2_data, fmo2_cutoff_data, disp, 0.5))
fmo2_cutoff_table.append(find_R_NuDimer_cutoff(fmo2_data, fmo2_cutoff_data, disp, 0.1))

fmo2_cutoff_table = pd.DataFrame(fmo2_cutoff_table)
fmo2_cutoff_table = fmo2_cutoff_table[["Title", "Rcutoff", "NuDimer"]]
fmo2_cutoff_table

,Title,Rcutoff,NuDimer
0,FMO2-hf-1.0,1.94,46
1,FMO2-hf-0.5,2.46,65
2,FMO2-hf-0.1,3.29,85
3,FMO2-mp2-1.0,2.73,69
4,FMO2-mp2-0.5,2.84,77
5,FMO2-mp2-0.1,3.39,87
6,FMO2-disp-1.0,2.74,73
7,FMO2-disp-0.5,2.84,77
8,FMO2-disp-0.1,3.39,87


# THREE-BODY Effect

* _Calculate the three-body energies of each trimer_<br/>
  _Then save them to a new column called E-xxx-delFMO3 in the data frame_

* _Determine the benchmark value (Ebm) for the energies of the three-body interaction ($\Delta$FMO3)_

In [ ]:
E_HF_delFMO3 = [ row.hf + row.coul_bath for index, row in fmo3_data.iterrows() ]
E_MP2_delFMO3 = [ row.mp2 + row.coul_bath for index, row in fmo3_data.iterrows() ]
E_disp_delFMO3 = [ row.mp2 - row.hf for index, row in fmo3_data.iterrows() ]

fmo3_data["E_HF_delFMO3"] = E_HF_delFMO3
fmo3_data["E_MP2_delFMO3"] = E_MP2_delFMO3
fmo3_data["E_disp_delFMO3"] = E_disp_delFMO3

Ebm_HF_delFMO3 = fmo3_data["E_HF_delFMO3"].sum()
Ebm_MP2_delFMO3 = fmo3_data["E_MP2_delFMO3"].sum()
Ebm_disp_delFMO3 = fmo3_data["E_disp_delFMO3"].sum()

* _Make a new data frame with the dimer index (out of I and J monomers) and R values from the FMO2 data frame_

In [ ]:
dimer = [ str(row.I) + "-" + str(row.J) for index, row in fmo2_data.iterrows() ]
dimer_R_ref_data = pd.DataFrame({"Dimer": dimer, "R": fmo2_data["R"]})

* _Create new columns for indexing the dimers IJ, IK and JK of FMO3 data table_

In [ ]:
IJ = [ str(row.I) + "-" + str(row.J) for index, row in fmo3_data.iterrows() ]
IK = [ str(row.I) + "-" + str(row.K) for index, row in fmo3_data.iterrows() ]
JK = [ str(row.J) + "-" + str(row.K) for index, row in fmo3_data.iterrows() ]
fmo3_data["IJ"] = IJ
fmo3_data["IK"] = IK 
fmo3_data["JK"] = JK

Three new columns for three ranges: 
* R_IJ
* R_IK 
* R_JK

They correspond to three columns IJ, IK and JK. <br/>
For each value of each column, look up this value in the dimer_R_reef_data, and take out the R value. 

In [33]:
%%time

index_array = list(dimer_R_ref_data["Dimer"])

R_IJ = []
R_IK = []
R_JK = []

for q in fmo3_data["IJ"]:
    index = index_array.index(q)
    R_value = dimer_R_ref_data.iloc[index]["R"]
    R_IJ.append(R_value)

for r in fmo3_data["IK"]:
    index = index_array.index(r)
    R_value = dimer_R_ref_data.iloc[index]["R"]
    R_IK.append(R_value)
    
for s in fmo3_data["JK"]:
    index = index_array.index(s)
    R_value = dimer_R_ref_data.iloc[index]["R"]
    R_JK.append(R_value)

CPU times: user 175 ms, sys: 3.42 ms, total: 179 ms
Wall time: 179 ms


In [34]:
fmo3_data["R_IJ"] = R_IJ
fmo3_data["R_IK"] = R_IK
fmo3_data["R_JK"] = R_JK

* Compare R_IJ, R_IK and R_JK <br/>
  _Sort them from lowest to highest values (R1, R2 and R3)_

In [35]:
R1 = []    # The lowest R value
R2 = []    # The medium R value
R3 = []    # The largest R value 

for index, row in fmo3_data.iterrows():
    compare_R = []
    compare_R.append(row.R_IJ)
    compare_R.append(row.R_IK)
    compare_R.append(row.R_JK)
    compare_R.sort()
    R1.append(compare_R[0])
    R2.append(compare_R[1])
    R3.append(compare_R[2])

fmo3_data["R1"] = R1
fmo3_data["R2"] = R2
fmo3_data["R3"] = R3

In [36]:
# Save the fmo2_data
fmo3_data.to_csv(fmo3_outfile)

### 3R MODEL

In [37]:
threeR_thres = R_thres    # R_threshold from fmo2_cutoff_data

E_HF_delFMO3_in_R = []
E_MP2_delFMO3_in_R = []
E_disp_delFMO3_in_R = []

for i in threeR_thres:
    
    E_HF_in_R = 0.0
    E_MP2_in_R = 0.0
    E_disp_in_R = 0.0
    
    for index, row in fmo3_data.iterrows():
        if row.R3 <= i:
            E_HF_in_R += row.E_HF_delFMO3
            E_MP2_in_R += row.E_MP2_delFMO3 
            E_disp_in_R += row.E_disp_delFMO3
            
    E_HF_delFMO3_in_R.append(E_HF_in_R)
    E_MP2_delFMO3_in_R.append(E_MP2_in_R)
    E_disp_delFMO3_in_R.append(E_disp_in_R)
    
diff_EHF = []
diff_EMP2 = []
diff_Edisp = []

for j in E_HF_delFMO3_in_R:
    diff_HF = (j - Ebm_HF_delFMO3)*2625.499638
    diff_HF = '{:.3f}'.format(diff_HF)
    diff_EHF.append(diff_HF)

for k in E_MP2_delFMO3_in_R:
    diff_MP2 = (k - Ebm_MP2_delFMO3)*2625.499638
    diff_MP2 = '{:.3f}'.format(diff_MP2)
    diff_EMP2.append(diff_MP2)
    
for l in E_disp_delFMO3_in_R: 
    diff_disp = (l - Ebm_disp_delFMO3)*2625.499638
    diff_disp = '{:.3f}'.format(diff_disp)
    diff_Edisp.append(diff_disp)

fmo3_3R_cutoff_data = pd.DataFrame({"R": threeR_thres, "diff_EHF": diff_EHF, "diff_EMP2": diff_EMP2, "diff_Edisp": diff_Edisp})
fmo3_3R_cutoff_data = fmo3_3R_cutoff_data.sort_values(by=["R"], ascending=False)

### 2R MODEL

In [38]:
twoR_thres = R_thres     # R_threshold from fmo2_cutoff_data

E_HF_delFMO3_in_R = []
E_MP2_delFMO3_in_R = []
E_disp_delFMO3_in_R = []

for i in twoR_thres:
    
    E_HF_in_R = 0.0
    E_MP2_in_R = 0.0
    E_disp_in_R = 0.0
    
    for index, row in fmo3_data.iterrows():
        if row.R2 <= i:
            E_HF_in_R += row.E_HF_delFMO3
            E_MP2_in_R += row.E_MP2_delFMO3 
            E_disp_in_R += row.E_disp_delFMO3
            
    E_HF_delFMO3_in_R.append(E_HF_in_R)
    E_MP2_delFMO3_in_R.append(E_MP2_in_R)
    E_disp_delFMO3_in_R.append(E_disp_in_R)
    
diff_EHF = []
diff_EMP2 = []
diff_Edisp = []

for j in E_HF_delFMO3_in_R:
    diff_HF = (j - Ebm_HF_delFMO3)*2625.499638
    diff_HF = '{:.3f}'.format(diff_HF)
    diff_EHF.append(diff_HF)

for k in E_MP2_delFMO3_in_R:
    diff_MP2 = (k - Ebm_MP2_delFMO3)*2625.499638
    diff_MP2 = '{:.3f}'.format(diff_MP2)
    diff_EMP2.append(diff_MP2)
    
for l in E_disp_delFMO3_in_R: 
    diff_disp = (l - Ebm_disp_delFMO3)*2625.499638
    diff_disp = '{:.3f}'.format(diff_disp)
    diff_Edisp.append(diff_disp)

fmo3_2R_cutoff_data = pd.DataFrame({"R": twoR_thres, "diff_EHF": diff_EHF, "diff_EMP2": diff_EMP2, "diff_Edisp": diff_Edisp})
fmo3_2R_cutoff_data = fmo3_2R_cutoff_data.sort_values(by=["R"], ascending=False)

### 1R MODEL

In [39]:
oneR_thres = R_thres     # R_threshold from fmo2_cutoff_data

E_HF_delFMO3_in_R = []
E_MP2_delFMO3_in_R = []
E_disp_delFMO3_in_R = []

for i in twoR_thres:
    
    E_HF_in_R = 0.0
    E_MP2_in_R = 0.0
    E_disp_in_R = 0.0
    
    for index, row in fmo3_data.iterrows():
        if row.R1 <= i:
            E_HF_in_R += row.E_HF_delFMO3
            E_MP2_in_R += row.E_MP2_delFMO3 
            E_disp_in_R += row.E_disp_delFMO3
            
    E_HF_delFMO3_in_R.append(E_HF_in_R)
    E_MP2_delFMO3_in_R.append(E_MP2_in_R)
    E_disp_delFMO3_in_R.append(E_disp_in_R)
    
diff_EHF = []
diff_EMP2 = []
diff_Edisp = []

for j in E_HF_delFMO3_in_R:
    diff_HF = (j - Ebm_HF_delFMO3)*2625.499638
    diff_HF = '{:.3f}'.format(diff_HF)
    diff_EHF.append(diff_HF)

for k in E_MP2_delFMO3_in_R:
    diff_MP2 = (k - Ebm_MP2_delFMO3)*2625.499638
    diff_MP2 = '{:.3f}'.format(diff_MP2)
    diff_EMP2.append(diff_MP2)
    
for l in E_disp_delFMO3_in_R: 
    diff_disp = (l - Ebm_disp_delFMO3)*2625.499638
    diff_disp = '{:.3f}'.format(diff_disp)
    diff_Edisp.append(diff_disp)

fmo3_1R_cutoff_data = pd.DataFrame({"R": twoR_thres, "diff_EHF": diff_EHF, "diff_EMP2": diff_EMP2, "diff_Edisp": diff_Edisp})
fmo3_1R_cutoff_data = fmo3_1R_cutoff_data.sort_values(by=["R"], ascending=False)

##### Define a function to find the R cutoff value according to the energy criterium
#### 3R Model

In [40]:
def find_R_NuTrimer_3R_cutoff(full_data, cutoff_data, energy_type, crit_value):
    dict_R_NuTrimer = {}
    
    if energy_type == "hf":
        R = []
        for index, row in cutoff_data.iterrows():
            if abs(float(row.diff_EHF)) > crit_value: 
                break 
            R.append(row.R)

        R_cutoff = R[-1]
        
        i = 0
        for index, row in full_data.iterrows():
            if row.R3 <= R_cutoff:
                i += 1        
        
    if energy_type == "mp2":
        R = []
        for index, row in cutoff_data.iterrows():
            if abs(float(row.diff_EMP2)) > crit_value: 
                break 
            R.append(row.R)

        R_cutoff = R[-1]
    
        i = 0
        for index, row in full_data.iterrows():
            if row.R3 <= R_cutoff:
                i += 1        
    
    if energy_type == "disp":
        R = []
        for index, row in cutoff_data.iterrows():
            if abs(float(row.diff_Edisp)) > crit_value: 
                break 
            R.append(row.R)

        R_cutoff = R[-1]
        
        i = 0
        for index, row in full_data.iterrows():
            if row.R3 <= R_cutoff:
                i += 1 
    
    dict_R_NuTrimer["Title"] = "FMO3-3R-" + energy_type + "-" + str(crit_value)
    dict_R_NuTrimer["Rcutoff"] = R_cutoff
    dict_R_NuTrimer["NuTrimer"] = i
    
    return dict_R_NuTrimer
########################################################
hf = "hf"
mp2 = "mp2"
disp = "disp"

#### 2R Model

In [41]:
def find_R_NuTrimer_2R_cutoff(full_data, cutoff_data, energy_type, crit_value):
    dict_R_NuTrimer = {}
    
    if energy_type == "hf":
        R = []
        for index, row in cutoff_data.iterrows():
            if abs(float(row.diff_EHF)) > crit_value: 
                break 
            R.append(row.R)

        R_cutoff = R[-1]
        
        i = 0
        for index, row in full_data.iterrows():
            if row.R2 <= R_cutoff:
                i += 1        
        
    if energy_type == "mp2":
        R = []
        for index, row in cutoff_data.iterrows():
            if abs(float(row.diff_EMP2)) > crit_value: 
                break 
            R.append(row.R)

        R_cutoff = R[-1]
    
        i = 0
        for index, row in full_data.iterrows():
            if row.R2 <= R_cutoff:
                i += 1        
    
    if energy_type == "disp":
        R = []
        for index, row in cutoff_data.iterrows():
            if abs(float(row.diff_Edisp)) > crit_value: 
                break 
            R.append(row.R)

        R_cutoff = R[-1]
        
        i = 0
        for index, row in full_data.iterrows():
            if row.R2 <= R_cutoff:
                i += 1 
    
    dict_R_NuTrimer["Title"] = "FMO3-2R-" + energy_type + "-" + str(crit_value)
    dict_R_NuTrimer["Rcutoff"] = R_cutoff
    dict_R_NuTrimer["NuTrimer"] = i
    
    return dict_R_NuTrimer
########################################################
hf = "hf"
mp2 = "mp2"
disp = "disp"

#### 1R Model

In [42]:
def find_R_NuTrimer_1R_cutoff(full_data, cutoff_data, energy_type, crit_value):
    dict_R_NuTrimer = {}
    
    if energy_type == "hf":
        R = []
        for index, row in cutoff_data.iterrows():
            if abs(float(row.diff_EHF)) > crit_value: 
                break 
            R.append(row.R)

        R_cutoff = R[-1]
        
        i = 0
        for index, row in full_data.iterrows():
            if row.R1 <= R_cutoff:
                i += 1        
        
    if energy_type == "mp2":
        R = []
        for index, row in cutoff_data.iterrows():
            if abs(float(row.diff_EMP2)) > crit_value: 
                break 
            R.append(row.R)

        R_cutoff = R[-1]
    
        i = 0
        for index, row in full_data.iterrows():
            if row.R1 <= R_cutoff:
                i += 1        
    
    if energy_type == "disp":
        R = []
        for index, row in cutoff_data.iterrows():
            if abs(float(row.diff_Edisp)) > crit_value: 
                break 
            R.append(row.R)

        R_cutoff = R[-1]
        
        i = 0
        for index, row in full_data.iterrows():
            if row.R1 <= R_cutoff:
                i += 1 
    
    dict_R_NuTrimer["Title"] = "FMO3-1R-" + energy_type + "-" + str(crit_value)
    dict_R_NuTrimer["Rcutoff"] = R_cutoff
    dict_R_NuTrimer["NuTrimer"] = i
    
    return dict_R_NuTrimer
########################################################
hf = "hf"
mp2 = "mp2"
disp = "disp"

3R Model cutoffs

In [43]:
fmo3_3r_cutoff_table = []
fmo3_3r_cutoff_table.append(find_R_NuTrimer_3R_cutoff(fmo3_data, fmo3_3R_cutoff_data, hf, 1.0))
fmo3_3r_cutoff_table.append(find_R_NuTrimer_3R_cutoff(fmo3_data, fmo3_3R_cutoff_data, hf, 0.5))
fmo3_3r_cutoff_table.append(find_R_NuTrimer_3R_cutoff(fmo3_data, fmo3_3R_cutoff_data, hf, 0.1))
fmo3_3r_cutoff_table.append(find_R_NuTrimer_3R_cutoff(fmo3_data, fmo3_3R_cutoff_data, mp2, 1.0))
fmo3_3r_cutoff_table.append(find_R_NuTrimer_3R_cutoff(fmo3_data, fmo3_3R_cutoff_data, mp2, 0.5))
fmo3_3r_cutoff_table.append(find_R_NuTrimer_3R_cutoff(fmo3_data, fmo3_3R_cutoff_data, mp2, 0.1))
fmo3_3r_cutoff_table.append(find_R_NuTrimer_3R_cutoff(fmo3_data, fmo3_3R_cutoff_data, disp, 1.0))
fmo3_3r_cutoff_table.append(find_R_NuTrimer_3R_cutoff(fmo3_data, fmo3_3R_cutoff_data, disp, 0.5))
fmo3_3r_cutoff_table.append(find_R_NuTrimer_3R_cutoff(fmo3_data, fmo3_3R_cutoff_data, disp, 0.1))

fmo3_3r_cutoff_table = pd.DataFrame(fmo3_3r_cutoff_table)
fmo3_3r_cutoff_table = fmo3_3r_cutoff_table[["Title", "Rcutoff", "NuTrimer"]]
fmo3_3r_cutoff_table

,Title,Rcutoff,NuTrimer
0,FMO3-3R-hf-1.0,1.78,44
1,FMO3-3R-hf-0.5,2.20,68
2,FMO3-3R-hf-0.1,2.29,106
3,FMO3-3R-mp2-1.0,2.42,130
4,FMO3-3R-mp2-0.5,2.73,170
5,FMO3-3R-mp2-0.1,2.84,228
6,FMO3-3R-disp-1.0,2.42,130
7,FMO3-3R-disp-0.5,2.73,170
8,FMO3-3R-disp-0.1,3.12,244


2R Model cutoffs

In [44]:
fmo3_2r_cutoff_table = []
fmo3_2r_cutoff_table.append(find_R_NuTrimer_2R_cutoff(fmo3_data, fmo3_2R_cutoff_data, hf, 1.0))
fmo3_2r_cutoff_table.append(find_R_NuTrimer_2R_cutoff(fmo3_data, fmo3_2R_cutoff_data, hf, 0.5))
fmo3_2r_cutoff_table.append(find_R_NuTrimer_2R_cutoff(fmo3_data, fmo3_2R_cutoff_data, hf, 0.1))
fmo3_2r_cutoff_table.append(find_R_NuTrimer_2R_cutoff(fmo3_data, fmo3_2R_cutoff_data, mp2, 1.0))
fmo3_2r_cutoff_table.append(find_R_NuTrimer_2R_cutoff(fmo3_data, fmo3_2R_cutoff_data, mp2, 0.5))
fmo3_2r_cutoff_table.append(find_R_NuTrimer_2R_cutoff(fmo3_data, fmo3_2R_cutoff_data, mp2, 0.1))
fmo3_2r_cutoff_table.append(find_R_NuTrimer_2R_cutoff(fmo3_data, fmo3_2R_cutoff_data, disp, 1.0))
fmo3_2r_cutoff_table.append(find_R_NuTrimer_2R_cutoff(fmo3_data, fmo3_2R_cutoff_data, disp, 0.5))
fmo3_2r_cutoff_table.append(find_R_NuTrimer_2R_cutoff(fmo3_data, fmo3_2R_cutoff_data, disp, 0.1))

fmo3_2r_cutoff_table = pd.DataFrame(fmo3_2r_cutoff_table)
fmo3_2r_cutoff_table = fmo3_2r_cutoff_table[["Title", "Rcutoff", "NuTrimer"]]
fmo3_2r_cutoff_table

,Title,Rcutoff,NuTrimer
0,FMO3-2R-hf-1.0,1.13,80
1,FMO3-2R-hf-0.5,1.13,80
2,FMO3-2R-hf-0.1,1.16,128
3,FMO3-2R-mp2-1.0,1.16,128
4,FMO3-2R-mp2-0.5,1.16,128
5,FMO3-2R-mp2-0.1,2.25,226
6,FMO3-2R-disp-1.0,1.16,128
7,FMO3-2R-disp-0.5,1.16,128
8,FMO3-2R-disp-0.1,1.94,168


1R Model cutoffs

In [45]:
fmo3_1r_cutoff_table = []
fmo3_1r_cutoff_table.append(find_R_NuTrimer_1R_cutoff(fmo3_data, fmo3_1R_cutoff_data, hf, 1.0))
fmo3_1r_cutoff_table.append(find_R_NuTrimer_1R_cutoff(fmo3_data, fmo3_1R_cutoff_data, hf, 0.5))
fmo3_1r_cutoff_table.append(find_R_NuTrimer_1R_cutoff(fmo3_data, fmo3_1R_cutoff_data, hf, 0.1))
fmo3_1r_cutoff_table.append(find_R_NuTrimer_1R_cutoff(fmo3_data, fmo3_1R_cutoff_data, mp2, 1.0))
fmo3_1r_cutoff_table.append(find_R_NuTrimer_1R_cutoff(fmo3_data, fmo3_1R_cutoff_data, mp2, 0.5))
fmo3_1r_cutoff_table.append(find_R_NuTrimer_1R_cutoff(fmo3_data, fmo3_1R_cutoff_data, mp2, 0.1))
fmo3_1r_cutoff_table.append(find_R_NuTrimer_1R_cutoff(fmo3_data, fmo3_1R_cutoff_data, disp, 1.0))
fmo3_1r_cutoff_table.append(find_R_NuTrimer_1R_cutoff(fmo3_data, fmo3_1R_cutoff_data, disp, 0.5))
fmo3_1r_cutoff_table.append(find_R_NuTrimer_1R_cutoff(fmo3_data, fmo3_1R_cutoff_data, disp, 0.1))

fmo3_1r_cutoff_table = pd.DataFrame(fmo3_1r_cutoff_table)
fmo3_1r_cutoff_table = fmo3_1r_cutoff_table[["Title", "Rcutoff", "NuTrimer"]]
fmo3_1r_cutoff_table

,Title,Rcutoff,NuTrimer
0,FMO3-1R-hf-1.0,1.13,208
1,FMO3-1R-hf-0.5,1.13,208
2,FMO3-1R-hf-0.1,1.16,272
3,FMO3-1R-mp2-1.0,1.13,208
4,FMO3-1R-mp2-0.5,1.13,208
5,FMO3-1R-mp2-0.1,1.16,272
6,FMO3-1R-disp-1.0,1.13,208
7,FMO3-1R-disp-0.5,1.16,272
8,FMO3-1R-disp-0.1,1.16,272
